In [ ]:
train_root_path = '/hpi/fs00/home/nils.strassenburg/datasets/coco/train2017'
train_annotations = '/hpi/fs00/home/nils.strassenburg/datasets/coco/annotations/instances_train2017.json'

test_root_path = '/hpi/fs00/home/nils.strassenburg/datasets/coco/test2017'
test_annotations = '/hpi/fs00/home/nils.strassenburg/datasets/coco/annotations/instances_test2017.json'

val_root_path = '/hpi/fs00/home/nils.strassenburg/datasets/coco/val2017'
val_annotations = '/hpi/fs00/home/nils.strassenburg/datasets/coco/annotations/instances_val2017.json'


